In [1]:
"""
References:
https://arxiv.org/pdf/1512.03385.pdf
https://arxiv.org/pdf/1604.04112v4.pdf
https://github.com/yu4u/cutout-random-erasing
"""
import numpy as np
from tensorflow.keras.layers import Add, Dense, Conv2D, BatchNormalization
from tensorflow.keras.layers import Activation, AveragePooling2D, Input, Flatten
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.regularizers import l2
from tensorflow.keras.models import Model

In [2]:
batch_size = 128
epochs = 180
n_classes = 10
learning_rate = 0.1

In [3]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

y_train = to_categorical(y_train, n_classes)
y_test = to_categorical(y_test, n_classes)

In [4]:
class Resnet:
    def __init__(self, size=44, stacks=3, starting_filter=16):
        self.size = size
        self.stacks = stacks
        self.starting_filter = starting_filter
        self.residual_blocks = (size - 2) // 6
        
    def get_model(self, input_shape=(32, 32, 3), n_classes=10):
        n_filters = self.starting_filter

        inputs = Input(shape=input_shape)
        network = self.layer(inputs, n_filters)
        network = self.stack(network, n_filters, True)

        for _ in range(self.stacks - 1):
            n_filters *= 2
            network = self.stack(network, n_filters)

        network = Activation('elu')(network)
        network = AveragePooling2D(pool_size=network.shape[1])(network)
        network = Flatten()(network)
        outputs = Dense(n_classes, activation='softmax', 
                        kernel_initializer='he_normal')(network)

        model = Model(inputs=inputs, outputs=outputs)

        return model
    
    def stack(self, inputs, n_filters, first_stack=False):
        stack = inputs

        if first_stack:
            stack = self.identity_block(stack, n_filters)
        else:
            stack = self.convolution_block(stack, n_filters)

        for _ in range(self.residual_blocks - 1):
            stack = self.identity_block(stack, n_filters)

        return stack
    
    def identity_block(self, inputs, n_filters):
        shortcut = inputs

        block = self.layer(inputs, n_filters, normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        block = Add()([shortcut, block])

        return block

    def convolution_block(self, inputs, n_filters, strides=2):
        shortcut = inputs

        block = self.layer(inputs, n_filters, strides=strides,
                           normalize_batch=False)
        block = self.layer(block, n_filters, activation=None)

        shortcut = self.layer(shortcut, n_filters,
                              kernel_size=1, strides=strides,
                              activation=None)

        block = Add()([shortcut, block])

        return block
    
    def layer(self, inputs, n_filters, kernel_size=3,
              strides=1, activation='elu', normalize_batch=True):
    
        convolution = Conv2D(n_filters, kernel_size=kernel_size,
                             strides=strides, padding='same',
                             kernel_initializer="he_normal",
                             kernel_regularizer=l2(1e-4))

        x = convolution(inputs)

        if normalize_batch:
            x = BatchNormalization()(x)

        if activation is not None:
            x = Activation(activation)(x)

        return x
    
def learning_rate_schedule(epoch):
    new_learning_rate = learning_rate

    if epoch <= 90:
        pass
    elif epoch > 90 and epoch <= 140:
        new_learning_rate = learning_rate * 0.1
    else:
        new_learning_rate = learning_rate * 0.01
        
    print('Learning rate:', new_learning_rate)
    
    return new_learning_rate

def get_random_eraser(p=0.5, s_l=0.02, s_h=0.4, r_1=0.3, r_2=1/0.3, v_l=0, v_h=255, pixel_level=False):
    def eraser(input_img):
        if input_img.ndim == 3:
            img_h, img_w, img_c = input_img.shape
        elif input_img.ndim == 2:
            img_h, img_w = input_img.shape

        p_1 = np.random.rand()

        if p_1 > p:
            return input_img

        while True:
            s = np.random.uniform(s_l, s_h) * img_h * img_w
            r = np.random.uniform(r_1, r_2)
            w = int(np.sqrt(s / r))
            h = int(np.sqrt(s * r))
            left = np.random.randint(0, img_w)
            top = np.random.randint(0, img_h)

            if left + w <= img_w and top + h <= img_h:
                break

        if pixel_level:
            if input_img.ndim == 3:
                c = np.random.uniform(v_l, v_h, (h, w, img_c))
            if input_img.ndim == 2:
                c = np.random.uniform(v_l, v_h, (h, w))
        else:
            c = np.random.uniform(v_l, v_h)

        input_img[top:top + h, left:left + w] = c

        return input_img

    return eraser

In [5]:
resnet = Resnet()

model = resnet.get_model()

optimizer = SGD(learning_rate=learning_rate, momentum=0.9)
model.compile(loss='categorical_crossentropy',
              optimizer=optimizer, metrics=['accuracy'])

model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [6]:
lr_scheduler = LearningRateScheduler(learning_rate_schedule)
callbacks = [lr_scheduler]

datagen = ImageDataGenerator(width_shift_range=4,
                             height_shift_range=4,
                             horizontal_flip=True,
                             preprocessing_function=get_random_eraser(p=1, pixel_level=True))
datagen.fit(x_train)

model.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
          validation_data=(x_test, y_test),
          epochs=epochs, workers=4,
          callbacks=callbacks)

model.save('model.h5')

Epoch 1/180
Learning rate: 0.1
391/391 [==============================] - 30s 62ms/step - loss: 2.0053 - accuracy: 0.3974 - val_loss: 1.6091 - val_accuracy: 0.5333
Epoch 2/180
Learning rate: 0.1
391/391 [==============================] - 26s 65ms/step - loss: 1.5737 - accuracy: 0.5437 - val_loss: 1.6287 - val_accuracy: 0.5537
Epoch 3/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 1.3549 - accuracy: 0.6206 - val_loss: 1.2497 - val_accuracy: 0.6726
Epoch 4/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 1.2031 - accuracy: 0.6727 - val_loss: 1.2575 - val_accuracy: 0.6657
Epoch 5/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 1.1053 - accuracy: 0.7038 - val_loss: 0.9940 - val_accuracy: 0.7425
Epoch 6/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 1.0329 - accuracy: 0.7287 - val_loss: 1.0013 - val_accuracy: 0.7384
Epoch 7/180
Lear

391/391 [==============================] - 26s 66ms/step - loss: 0.6402 - accuracy: 0.8689 - val_loss: 0.6161 - val_accuracy: 0.8846
Epoch 51/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 0.6409 - accuracy: 0.8681 - val_loss: 0.6798 - val_accuracy: 0.8635
Epoch 52/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 0.6313 - accuracy: 0.8747 - val_loss: 0.7897 - val_accuracy: 0.8251
Epoch 53/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 0.6359 - accuracy: 0.8701 - val_loss: 0.8079 - val_accuracy: 0.8258
Epoch 54/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 0.6296 - accuracy: 0.8728 - val_loss: 0.8144 - val_accuracy: 0.8262
Epoch 55/180
Learning rate: 0.1
391/391 [==============================] - 25s 64ms/step - loss: 0.6281 - accuracy: 0.8755 - val_loss: 1.3317 - val_accuracy: 0.6330
Epoch 56/180
Learning rate: 0.1
391/391 [=

391/391 [==============================] - 27s 68ms/step - loss: 0.4064 - accuracy: 0.9461 - val_loss: 0.4398 - val_accuracy: 0.9389
Epoch 100/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 26s 65ms/step - loss: 0.3969 - accuracy: 0.9492 - val_loss: 0.4344 - val_accuracy: 0.9399
Epoch 101/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 26s 66ms/step - loss: 0.3933 - accuracy: 0.9484 - val_loss: 0.4339 - val_accuracy: 0.9388
Epoch 102/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 27s 69ms/step - loss: 0.3835 - accuracy: 0.9506 - val_loss: 0.4302 - val_accuracy: 0.9407
Epoch 103/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 28s 72ms/step - loss: 0.3797 - accuracy: 0.9512 - val_loss: 0.4262 - val_accuracy: 0.9401
Epoch 104/180
Learning rate: 0.010000000000000002
391/391 [==============================] - 28s 71ms/step - loss: 0.3713 - accuracy: 0

391/391 [==============================] - 26s 66ms/step - loss: 0.2296 - accuracy: 0.9734 - val_loss: 0.3410 - val_accuracy: 0.9465
Epoch 145/180
Learning rate: 0.001
391/391 [==============================] - 27s 67ms/step - loss: 0.2265 - accuracy: 0.9742 - val_loss: 0.3374 - val_accuracy: 0.9469
Epoch 146/180
Learning rate: 0.001
391/391 [==============================] - 26s 65ms/step - loss: 0.2253 - accuracy: 0.9736 - val_loss: 0.3376 - val_accuracy: 0.9476
Epoch 147/180
Learning rate: 0.001
391/391 [==============================] - 27s 69ms/step - loss: 0.2237 - accuracy: 0.9741 - val_loss: 0.3370 - val_accuracy: 0.9471
Epoch 148/180
Learning rate: 0.001
391/391 [==============================] - 26s 65ms/step - loss: 0.2208 - accuracy: 0.9757 - val_loss: 0.3391 - val_accuracy: 0.9464
Epoch 149/180
Learning rate: 0.001
391/391 [==============================] - 26s 65ms/step - loss: 0.2242 - accuracy: 0.9746 - val_loss: 0.3377 - val_accuracy: 0.9478
Epoch 150/180
Learning rate

D:\anaconda\envs\acnn\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '
